# Git Repo Cleanup: Entfernen großer Dateien mit BFG

Dieses Notebook beschreibt Schritt für Schritt, wie du versehentlich committete große Dateien (z.\u00a0B. JARs oder den `target/`-Ordner) aus deiner Git-Historie entfernen kannst.

## 1. Ausgangslage
- Du hast gro\u00dfe Dateien (z.B. JARs) in dein Repo committet.
- Ein `git push` h\u00e4ngt, das Repo ist zu gro\u00df.
- Auch nach L\u00f6schen und .gitignore sind die Daten in der History und bremsen alles aus.

## 2. Die schlimmsten Dateien finden
Das hilft dir zu sehen, was die Repo-Gr\u00f6\u00dfe treibt:
```bash
git rev-list --objects --all | \
  git cat-file --batch-check='%(objecttype) %(objectname) %(objectsize) %(rest)' | \
  grep ^blob | sort -k3nr | head -10
```
Das gibt dir die gr\u00f6\u00dften Dateien mit SHA und Pfad.

## 3. .gitignore anpassen
Damit du nie wieder versehentlich solche Dateien ins Repo bekommst:
```bash
echo "target/" >> .gitignore
echo "*.jar"   >> .gitignore
git add .gitignore
git commit -m "Add target/ and *.jar to .gitignore"
```
Optional: L\u00f6sche schon getrackte Dateien aus dem Repo:
```bash
git rm --cached -r target/
git rm --cached *.jar
git commit -m "Remove binaries and target folder from repo"
```

## 4. BFG Repo-Cleaner installieren
Auf macOS (Homebrew):
```bash
brew install bfg
```
Oder von https://rtyley.github.io/bfg-repo-cleaner/

## 5. BFG Repo Cleaner ausf\u00fchren
Dateien wie JARs aus der gesamten Git-Historie entfernen:
```bash
bfg --delete-files "*.jar"
```
Build-Ordner wie `target/` komplett entfernen:
```bash
bfg --delete-folders "target" --no-blob-protection
```
Danach **Git-Garbage-Collection** ausf\u00fchren:
```bash
git reflog expire --expire=now --all && git gc --prune=now --aggressive
```

> \u274c **Warnung:** Danach ist die History ge\u00e4ndert \u2013 du musst mit `--force` pushen und alle Entwickler brauchen ein frisches Clone!

## 6. Force Push zum Bereinigen des Remotes
```bash
git push --force
```

Fertig! Die dicken Dateien sind wirklich raus.

## 7. Nacharbeiten: Lokale Repos von anderen Entwicklern
**Warnung:** Wer das Repo lokal schon hat, sollte jetzt am besten einmal neu clonen:
```bash
mv dein-altes-repo dein-altes-repo_backup
git clone git@github.com:deine-org/dein-repo.git
```
Falls jemand trotzdem "weiterarbeitet":
```bash
git fetch
git reset --hard origin/main
```

## 8. Git LFS \u2013 Besser f\u00fcr Bin\u00e4rdaten
- F\u00fcr Bin\u00e4rdaten wie JARs, Bilder, Modelle etc. **immer Git LFS verwenden**.
- Siehe: https://git-lfs.github.com/
- Installieren:
    ```bash
    brew install git-lfs
    git lfs install
    git lfs track "*.jar"
    git add .gitattributes
    ```
- Gro\u00dfe Dateien landen dann **nicht mehr** in der Git-History.

# Normaler Git → GitHub Workflow via CLI

## 1. Initialisierung eines neuen Git-Repositories (falls noch nicht geschehen)
```bash
git init
git add .
git commit -m "Initial commit"
```

## 2. Remote Repository hinzufügen
```bash
git remote add origin git@github.com:<Organisation>/<Repository>.git
```
Ersetze `<Organisation>` und `<Repository>` durch deine Werte.

## 3. Push zum Remote und Upstream setzen
```bash
git push -u origin main
```

## 4. Arbeiten an der Codebasis
Änderungen hinzufügen, committen und pushen:
```bash
git add <datei/ordner>
git commit -m "Beschreibung der Änderungen"
git push
```

## 5. Von Remote Änderungen holen und mergen
```bash
git pull
```

## 6. Branch wechseln oder neuen Branch anlegen
```bash
git checkout -b feature/mein-feature
```

## 7. SSH-Key Check (optional, falls Authentifizierungsprobleme)
```bash
ssh -T git@github.com
```
Meldung sollte sein: "Hi username! You've successfully authenticated..."

## 8. SSH Key mit GitHub verknüpfen (falls noch nicht geschehen)
- SSH-Key generieren:
  ```bash
  ssh-keygen -t ed25519 -C "deine.email@domain.com"
  ```
- Öffne den Inhalt von `~/.ssh/id_ed25519.pub` und füge den Key in GitHub unter `Settings -> SSH and GPG keys` hinzu.

---

**Fertig!** Damit hast du alle typischen Git- und GitHub-CLI-Befehle beisammen.